In [1]:
import json
import os

import requests
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1", api_key=os.getenv("GROQ_API_KEY")
)

In [3]:
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    print(data["current"])
    return data["current"]

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather data like temperature, wind speed for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [5]:
system_prompt = (
    "You are a helpful weather assistant who nicely answers in full sentences."
)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the temperature like in Kolkata today?"},
]

In [6]:
completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [469]:
# completion.model_dump()

In [7]:
completion.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zw94', function=Function(arguments='{"latitude": 23.5293, "longitude": 88.3639}', name='get_weather'), type='function')]))

In [8]:
if(completion.choices[0].finish_reason == "stop"):
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        tools=tools,
    )

In [9]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)


def checkforToolCalls(completion):
    if completion.choices[0].message.tool_calls:
        print("Tool call found")
        for tool_call in completion.choices[0].message.tool_calls:
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            messages.append(completion.choices[0].message)
            result = call_function(name, args)
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result),
                }
            )

In [10]:
checkforToolCalls(completion)

Tool call found
{'time': '2025-02-11T18:15', 'interval': 900, 'temperature_2m': 19.8, 'wind_speed_10m': 6.2}


In [11]:
messages

[{'role': 'system',
  'content': 'You are a helpful weather assistant who nicely answers in full sentences.'},
 {'role': 'user', 'content': "What's the temperature like in Kolkata today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zw94', function=Function(arguments='{"latitude": 23.5293, "longitude": 88.3639}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_zw94',
  'content': '{"time": "2025-02-11T18:15", "interval": 900, "temperature_2m": 19.8, "wind_speed_10m": 6.2}'}]

In [12]:
completion_2 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [13]:
final_response = completion_2.choices[0].message.content
final_response

'The current temperature in Kolkata is around 19.8°C and there is a gentle wind of 6.2 m/s.'

In [14]:
messages.append(completion_2.choices[0].message)

In [15]:
messages

[{'role': 'system',
  'content': 'You are a helpful weather assistant who nicely answers in full sentences.'},
 {'role': 'user', 'content': "What's the temperature like in Kolkata today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zw94', function=Function(arguments='{"latitude": 23.5293, "longitude": 88.3639}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_zw94',
  'content': '{"time": "2025-02-11T18:15", "interval": 900, "temperature_2m": 19.8, "wind_speed_10m": 6.2}'},
 ChatCompletionMessage(content='The current temperature in Kolkata is around 19.8°C and there is a gentle wind of 6.2 m/s.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)]

In [16]:
messages.append(
    {"role": "user", "content": "What is the temperature of Kolkata in Fahrenheit?"}
)

In [17]:
completion_3 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [18]:
completion_3.choices[0].message.content

'To convert the temperature to Fahrenheit, we need to use a conversion function. However, the conversion function is not available in the current knowledge base. But based on the given answer:\n\nThe current temperature in Kolkata is around 19.8°C and there is a gentle wind of 6.2 m/s.\n\nWe can manually perform the conversion as follows:\nTemperature in Fahrenheit = (Temperature in Celsius × 9/5) + 32\nTemperature in Fahrenheit = (19.8 × 9/5) + 32\nTemperature in Fahrenheit = (177.2/5) + 32\nTemperature in Fahrenheit = 35.44 + 32\nTemperature in Fahrenheit = 67.5\n\nSo, the current temperature in Kolkata is around 67.5°F.'